In [1]:
!pip install scikit-learn==1.0.0
!pip install wget==3.2
!pip install gensim==3.6.0
!pip install psutil==5.4.8
!pip install spacy==2.2.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.corpus import state_union



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import wget
import gzip
import shutil

In [5]:
gn_vec_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"
if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"):
    if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin"):
        #Downloading the reqired model
        if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"):
            if not os.path.exists("/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"):
                wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
            gn_vec_zip_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"
        else:
            gn_vec_zip_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin.gz"
        #Extracting the required model
        with gzip.open(gn_vec_zip_path, 'rb') as f_in:
            with open(gn_vec_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        gn_vec_path = "/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin" + gn_vec_path

print(f"Model at {gn_vec_path}")

Model at /content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin


In [6]:
import warnings #This module ignores the various types of warnings generated
warnings.filterwarnings("ignore") 

import psutil #This module helps in retrieving information on running processes and system resource utilization
process = psutil.Process(os.getpid())
from psutil import virtual_memory
mem = virtual_memory()

import time #This module is used to calculate the time  

In [7]:
from gensim.models import Word2Vec, KeyedVectors
pretrainedpath = gn_vec_path

#Load W2V model. This will take some time, but it is a one time effort! 
pre = process.memory_info().rss
print("Memory used in GB before Loading the Model: %0.2f"%float(pre/(10**9))) #Check memory usage before loading the model
print('-'*10)

start_time = time.time() #Start the timer
ttl = mem.total #Toal memory available

model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True) #load the model
print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

post = process.memory_info().rss
print("Memory used in GB after Loading the Model: {:.2f}".format(float(post/(10**9)))) #Calculate the memory used after loading the model
print('-'*10)

print("Percentage increase in memory usage: {:.2f}% ".format(float((post/pre)*100))) #Percentage increase in memory after loading the model
print('-'*10)

print("Number of words in vocablulary: ",len(model.vocab)) #Number of words in the vocabulary. 

Memory used in GB before Loading the Model: 0.18
----------
56.06 seconds taken to load
----------
Finished loading Word2Vec
----------
Memory used in GB after Loading the Model: 5.06
----------
Percentage increase in memory usage: 2832.20% 
----------
Number of words in vocablulary:  3000000


In [8]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.bin', binary=True)  

In [9]:
my_trained_vectors = model.wv
# save vectors to file to use them later
my_trained_vectors.save_word2vec_format('/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.txt', fvocab='GoogleNews-vectors-negative300_vocab.txt', binary=False)
#my_trained_vectors.save_word2vec_format('/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.txt', binary=False)

In [10]:
train_file_path = '/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt'
val_file_path = '/content/drive/MyDrive/Event_Extraction/files/val_attn_sp.txt'
test_file_path = '/content/drive/MyDrive/Event_Extraction/files/test_attn_sp.txt'

relation_file_path = '/content/drive/MyDrive/Event_Extraction/files/relations.txt'
emb_google_txt = '/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.txt'
avg_vec_file = '/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300_avg_vec.txt'

data_all_file_path = "/content/drive/MyDrive/Event_Extraction/data/data_all.npy"

files = [train_file_path, val_file_path, test_file_path]

In [11]:
#stop_words = set(stopwords.words('english'))

In [12]:
label_to_int = {}
int_to_label= {} 

with open(relation_file_path, 'r') as f:
    for line in f: 
        line = line.strip().split()
        key = str(line[0])
        val = int(line[1])
        label_to_int[key] = val 
        int_to_label[val] = key 
        
print(int_to_label)
#with open("label_to_int.")

{0: 'Other', 1: 'Message-Topic(e1,e2)', 2: 'Message-Topic(e2,e1)', 3: 'Product-Producer(e1,e2)', 4: 'Product-Producer(e2,e1)', 5: 'Instrument-Agency(e1,e2)', 6: 'Instrument-Agency(e2,e1)', 7: 'Entity-Destination(e1,e2)', 8: 'Entity-Destination(e2,e1)', 9: 'Cause-Effect(e1,e2)', 10: 'Cause-Effect(e2,e1)', 11: 'Component-Whole(e1,e2)', 12: 'Component-Whole(e2,e1)', 13: 'Entity-Origin(e1,e2)', 14: 'Entity-Origin(e2,e1)', 15: 'Member-Collection(e1,e2)', 16: 'Member-Collection(e2,e1)', 17: 'Content-Container(e1,e2)', 18: 'Content-Container(e2,e1)'}


In [13]:
print(label_to_int)

{'Other': 0, 'Message-Topic(e1,e2)': 1, 'Message-Topic(e2,e1)': 2, 'Product-Producer(e1,e2)': 3, 'Product-Producer(e2,e1)': 4, 'Instrument-Agency(e1,e2)': 5, 'Instrument-Agency(e2,e1)': 6, 'Entity-Destination(e1,e2)': 7, 'Entity-Destination(e2,e1)': 8, 'Cause-Effect(e1,e2)': 9, 'Cause-Effect(e2,e1)': 10, 'Component-Whole(e1,e2)': 11, 'Component-Whole(e2,e1)': 12, 'Entity-Origin(e1,e2)': 13, 'Entity-Origin(e2,e1)': 14, 'Member-Collection(e1,e2)': 15, 'Member-Collection(e2,e1)': 16, 'Content-Container(e1,e2)': 17, 'Content-Container(e2,e1)': 18}


In [14]:
words_dataset = set()

def create_words_dataset():
    for f_name in files: 
        f = open(f_name, 'r')
        lines = f.readlines()
        f.close()
        
        lines = [ l.strip().split(" ")[2:]  for l in lines]
    
        for line in lines:
            for w in line:
                words_dataset.add(w)
    
    print("len(words_dataset)", len(words_dataset))
    
    with open('/content/drive/MyDrive/Event_Extraction/data/words_dataset.txt', 'w') as f: 
        for w in sorted(words_dataset):
            f.write(str(w) + "\n")
        print('/content/drive/MyDrive/Event_Extraction/data/words_dataset.txt created')
    
    
create_words_dataset()

len(words_dataset) 23449
/content/drive/MyDrive/Event_Extraction/data/words_dataset.txt created


In [15]:
#!python -m spacy download en_core_web_md


In [16]:
#import spacy

#%time 
#nlp = spacy.load('en_core_web_md')
# process a sentence using the model
#mydoc = nlp("Canada is a large country")
#Get a vector for individual words
#print(doc[0].vector) #vector for 'Canada', the first word in the text 
#print(mydoc.vector) #Averaged vector for the entire sentence

In [17]:
word_to_emb = {}

with open('/content/drive/MyDrive/Event_Extraction/embeddings/GoogleNews-vectors-negative300.txt', 'r', encoding='utf-8') as f: 
    first = True
    
    for line in f:
        if first == True:
            first = False
            continue
        line = line.strip().split()
        if len(line) != 301:
            continue 
        word = str(line[0])
        vec = [float(x) for x in line[1:]]
        vec = np.array(vec, dtype='float64')
        
        if word in words_dataset:
            word_to_emb[word] = vec
        elif word.lower() in words_dataset and word.lower() not in word_to_emb: 
            word_to_emb[word.lower()] = vec

In [18]:
def get_avg_vec(file_name):
    with open(file_name, 'r') as f:
        line = f.readline()
        line = line.strip().split()
        line = [float(x) for x in line]
        avg_vec = np.array(line, dtype='float64')
        print("avg_vec.shape", avg_vec.shape)
        return avg_vec

avg_vec = get_avg_vec(avg_vec_file)

avg_vec.shape (300,)


In [19]:
word_to_int = {}
embedding = []

unknown_words = 0
word_to_int['PADDING'] = len(word_to_int)
embedding.append(np.zeros(300, dtype='float64'))

for w in sorted(words_dataset): 
    word_to_int[w] = len(word_to_int)
    if w in word_to_emb:
        embedding.append(word_to_emb[w])
    elif w.lower() in word_to_emb:
        embedding.append(word_to_emb[w.lower()])
    else:
        unknown_words += 1
        embedding.append(avg_vec)

embedding = np.array(embedding, dtype='float64')
print("len(word_to_int)", len(word_to_int)) # 25656
print("embedding.shape", embedding.shape) # (25656, 300)
print("unknown_words", unknown_words) # 2652

len(word_to_int) 23450
embedding.shape (23450, 300)
unknown_words 2604


In [20]:
def get_max_sent_len(files):
    max_sent_len = 0 
    for fname in files: 
        f = open(fname, 'r')
        lines = f.readlines()
        f.close()
        for l in lines: 
            l = l.strip().split(" ")[2:]
            max_sent_len = max(max_sent_len, len(l))
    return max_sent_len

max_sent_len = get_max_sent_len(files)
print("max_sent_len", max_sent_len) # 102-1 = 101

max_sent_len 101


In [21]:
def create_matrices(file_name, word_to_int, label_to_int, max_sent_len):
    X = []
    Y = []
    
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    lines = [line.strip().split() for line in lines]
    
    for line in lines: 
        Y.append(label_to_int[line[1]])
        line = line[2:]
        tmp = np.zeros(max_sent_len, dtype='int32')
        for i in range(len(line)):
            tmp[i] = word_to_int[line[i]]
        X.append(tmp)

        
    X = np.array(X, dtype='int32')
    Y = np.array(Y, dtype='int32')
    
    return [X, Y]

In [22]:
train_set = create_matrices(train_file_path, word_to_int, label_to_int, max_sent_len)
val_set = create_matrices(val_file_path, word_to_int, label_to_int, max_sent_len)
test_set = create_matrices(test_file_path, word_to_int, label_to_int, max_sent_len)

In [24]:
data_all = [train_set, val_set, test_set, embedding]

np.save(data_all_file_path, data_all)